In [56]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swapn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\swapn\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [57]:
dataset = pd.read_csv('training_data.tsv', delimiter='\t')

In [3]:
type(dataset)

pandas.core.frame.DataFrame

In [4]:
print(dataset)

                                                   sent  \
0                                        Make remainder   
1             Set a reminder on date 23rd November'2016   
2                           I need a daily wake up call   
3                            remind me 6 pm today eveng   
4                    Hi Pls to make one reminder for me   
5                 Remind me at 28 December for recharge   
6                         Can u help me with reminder..   
7                   Can u pls remind me at 7pm on 8 Jan   
8                             What is my next reminder?   
9               Can u please set me an alarm at 7.30 am   
10    Set a reminder on 4 th Dec of going to meet so...   
11    Remind me to purchase shoe polish liquid Date:...   
12    Thanks at least I'll remember my loves birthda...   
13            I setd a reminder but it dint remained me   
14    Susan dmello meeting with sujit sir remind him...   
15    I need to set reminder to msg babbu at 7 in ev... 

In [43]:
rows = []
for i in range(0, 9819):
    review = re.sub('[^a-zA-Z0-9:/]', ' ', str(dataset['sent'][i]))
    review = review.lower()
    tokens = nltk.word_tokenize(review)
    tagged = nltk.pos_tag(tokens)
    rows.append([review,tokens,tagged])

In [44]:
for row in rows[:5]:
    print(row)

['make remainder', ['make', 'remainder'], [('make', 'VB'), ('remainder', 'NN')]]
["set a reminder on date 23rd november'2016", ['set', 'a', 'reminder', 'on', 'date', '23rd', "november'2016"], [('set', 'VB'), ('a', 'DT'), ('reminder', 'NN'), ('on', 'IN'), ('date', 'NN'), ('23rd', 'CD'), ("november'2016", 'NN')]]
['i need a daily wake up call', ['i', 'need', 'a', 'daily', 'wake', 'up', 'call'], [('i', 'NNS'), ('need', 'VBP'), ('a', 'DT'), ('daily', 'JJ'), ('wake', 'NN'), ('up', 'RP'), ('call', 'NN')]]
['remind me 6 pm today eveng', ['remind', 'me', '6', 'pm', 'today', 'eveng'], [('remind', 'VB'), ('me', 'PRP'), ('6', 'CD'), ('pm', 'NN'), ('today', 'NN'), ('eveng', 'VBP')]]
['hi pls to make one reminder for me', ['hi', 'pls', 'to', 'make', 'one', 'reminder', 'for', 'me'], [('hi', 'NN'), ('pls', 'NN'), ('to', 'TO'), ('make', 'VB'), ('one', 'CD'), ('reminder', 'NN'), ('for', 'IN'), ('me', 'PRP')]]


In [45]:
def phrase_extract(row):
    grammar = "NP: {<VB.*>?<RB>?<PRP.*>?<IN>?<DT>?<JJ.*>*<NN.*>+}"
    
    parser = nltk.RegexpParser(grammar)
    
    tree = parser.parse(row[2])
    a = [s for s in tree.subtrees() if s.label() == "NP"]
    c = []
    num = []
    
    # These keywords were not included as label in training dat, so don't consider here also
    key  = ["monday","tuesday", "wednesday", "thursday","friday","saturday","sunday","today","tomorrow","yesterday", "reminder", "remind", "th", "pm","am"]
    
    for i in range(len(a)):
        count=0
        phrase = ""
        for j in range(len(a[i])):
            if a[i][j][0].lower() in key:
                phrase = phrase
            else :
                phrase = phrase + str(a[i][j][0]) + " "
                count = count+1
        c.append(phrase)
        num.append(count)
        #print (c)
        #print (num)
    
    if(c==[] or max(num)<=1):
        return "Not Found"
    else :
        maxi = max(num)
        for i in range(len(num)):
            if(num[i]==maxi):
                return c[i].rstrip()

In [52]:
print("Reminder: ", dataset['sent'][15]) 
print("Phrase: ", phrase_extract(rows[15]))

Reminder:  I need to set reminder to msg babbu at 7 in evening
Phrase:  msg babbu


In [53]:
eval_dataset = []
# Reading file line by line 
with open("eval_data.txt", 'r+') as eval_data:
    for line in eval_data:
        string = re.sub('[^a-zA-Z0-9:/]', ' ', str(line))
        string = string.lower()
        tokens = nltk.word_tokenize(string)
        tagged = nltk.pos_tag(tokens)
        eval_dataset.append([string,tokens,tagged])
        
    
print(eval_dataset)

[['reminder for daily reporting and attendance on mob app daily 9:00 am ', ['reminder', 'for', 'daily', 'reporting', 'and', 'attendance', 'on', 'mob', 'app', 'daily', '9:00', 'am'], [('reminder', 'NN'), ('for', 'IN'), ('daily', 'JJ'), ('reporting', 'NN'), ('and', 'CC'), ('attendance', 'NN'), ('on', 'IN'), ('mob', 'NN'), ('app', 'JJ'), ('daily', 'JJ'), ('9:00', 'CD'), ('am', 'VBP')]], ['how will you remind me let me know ', ['how', 'will', 'you', 'remind', 'me', 'let', 'me', 'know'], [('how', 'WRB'), ('will', 'MD'), ('you', 'PRP'), ('remind', 'VB'), ('me', 'PRP'), ('let', 'VB'), ('me', 'PRP'), ('know', 'VB')]], ['can u remind me till 11 ', ['can', 'u', 'remind', 'me', 'till', '11'], [('can', 'MD'), ('u', 'VB'), ('remind', 'VB'), ('me', 'PRP'), ('till', 'VB'), ('11', 'CD')]], ['call at 9 am every day ', ['call', 'at', '9', 'am', 'every', 'day'], [('call', 'NN'), ('at', 'IN'), ('9', 'CD'), ('am', 'VBP'), ('every', 'DT'), ('day', 'NN')]], ['remind me to bid for flights on 13th december at 

In [54]:
print("Reminder: ",eval_dataset[5][0])
print("Phrase: ", phrase_extract(eval_dataset[5]))

Reminder:  reminder 11am tomorrorw buy bell for papu 
Phrase:  tomorrorw buy bell


In [59]:
with open('eval_dataset.csv', mode='w', newline='') as csv_file:
    headers = ['sent', 'label']
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    
    writer.writeheader()
    for i in range(len(eval_dataset)):
        writer.writerow({'sent':eval_dataset[i][0],'label':phrase_extract(eval_dataset[i])})